Idea Explorer in a notebook.

In [ ]:
#!/usr/local/bin/python3.9 

import os
import re
import time
import glob
import pandas as pd
from datetime import datetime
from plistlib import load
from urllib.parse import urlparse
from collections import Counter
from collections import defaultdict

In [ ]:
def TheArchivePath():
    #  Variables that ultimately revel The Archive's plist file.
        bundle_id = "de.zettelkasten.TheArchive"
        team_id = "FRMDA3XRGC"
        fileName = os.path.expanduser(
            "~/Library/Group Containers/{0}.{1}.prefs/Library/Preferences/{0}.{1}.prefs.plist".format(team_id, bundle_id))
        with open(fileName, 'rb') as fp:
            pl = load(fp) # load is a special function for use with a plist
            path = urlparse(pl['archiveURL']) # 'archiveURL' is the key that pairs with the zk path
        return (path.path) # path is the part of the path that is formatted for use as a path.

#####
# Function for outgoing links with note titles
#####
def explorer(zettel):
    for key, value in zk_info.items():
        if value["ntitle"] == zettel:
            for i in value["outbound"]:
                if zk_info.get(i):
                    print(i, zk_info[i]["ntitle"],"\n", zk_info[i]["age"],"\n", zk_info[i]["Last Modified"],"\n", zk_info[i]["WC"], "words")
                    print("This is an outbound link.")
                    print(zk_info[i]["tags"])
                else:
                    print(i, "This is an intralink.")

#####
# Function for tag cloud
#####
def tag_cloud(zettel):  # sourcery skip: do-not-use-bare-except
    """
    This function returns a list of tags sorted by the prevalence of their occurrence.

    :param zettel: A string representing a specific record in the `zk_info` dictionary.
    :return: A list of related records to the `zettel`.
    """
    try: # Try to execute the following code block. Some records have empty 'tags' field, so this will throw an error.
        outbound_records = [zk_info[id] for id in zk_info[zettel]['outbound'] if 'outbound' in zk_info[id]]
        tag_count = defaultdict(int)
        for record in outbound_records:
            for tag_list in record['tags']:
                for tag in tag_list:
                    tag_count[tag] += 1
        sorted_tag_count = sorted(tag_count.items(), key=lambda x: x[1], reverse=True)
        for tag, count in sorted_tag_count:
            print(f"{tag} {count}")
            # return tag, count
    except: 
        # Do nothing, just continue execution
        pass
        
#####
# Function for inbound links uuids
#####        
def inbound_uuid(zettel):
    # sourcery skip: for-append-to-extend, inline-immediately-returned-variable, list-comprehension
    """
    This function returns a list of records related to the `zettel` by checking if the `zettel` is in the 'outbound' field of each record in the `zk_info` dictionary.

    :param zettel: A string representing a specific record in the `zk_info` dictionary.
    :return: A list of related records to the `zettel`.
    """
    related_records = []
    for key, value in zk_info.items():
        # If the record has an 'outbound' field and the `zettel` is in it, and the key is not 'outbound', append the key to the list of related records
        if 'outbound' in value and zettel in value['outbound'] and key != 'outbound':
            related_records.append(key)
    return related_records

#####
# Function for labeling links out, in, bidirectional
#####  
def label_links(outbound, inbound):
    # This function returns a sorted list of links, to and from the target zettel, with their directionality.
    combined = set(outbound + inbound)
    # set() removes duplicates
    direction = []
    for value in combined:
        if value == zettel:
            direction.append(f"{value} Originator")
        elif value in outbound and value in inbound:
            direction.append(f"{value} Bilateral")   
        elif value in outbound:
            direction.append(f"{value} Outbound")
        elif value in inbound:
            direction.append(f"{value} Inbound")

    direction.sort(key=lambda x: x.split(" ")[0], reverse=True)
    # sort by the first part of the string, which is the UUID, making the list sort with newest dates first
    return direction

#####
# Function for extracting days ago from last modified date
##### 
def extract_days_ago(s):
    pattern = r'(\d+) days ago'
    match = re.search(pattern, s)
    if match:
        return int(match.group(1))
    else:
        return 0  # or some default value if the pattern is not found

In [ ]:
print ("The Archive Path is: ", TheArchivePath())